## $$1 \ \ \textit{show data}$$

In [ ]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.pyplot import Axes, Figure
from matplotlib import font_manager
from matplotlib.font_manager import FontProperties

from einops import rearrange, repeat

basis_properties = ['x', 'v', 'properties']
properties_head = [

    "pe", # total potential energy
    "ke", # kinetic energy
    "etotal", # total energy (pe + ke)
    "evdwl", # van der Waals pairwise energy (includes etail)
    "ecoul", # Coulombic pairwise energy

    "epair", # pairwise energy (evdwl + ecoul + elong)
    "ebond", # bond energy
    "eangle", # angle energy
    "edihed", # dihedral energy
    "eimp", # improper energy
    
    "emol", # molecular energy (ebond + eangle + edihed + eimp)
    "elong", # long-range kspace energy
    "etail", # van der Waals energy long-range tail correction
    "enthalpy", # enthalpy (etotal + press*vol)
    "ecouple", # cumulative energy change due to thermo/baro statting fixes
    "econserve", # pe + ke + ecouple = etotal + ecouple
]


def draw(Canvas: Axes, row: int, col: int, dgroup: list | np.ndarray, labels: list, fontsize: FontProperties | None = None):
    for i in range(row):
        for j in range(col):
            if i * col + j > len(dgroup):
                break
            axe: Axes = Canvas[i, j]
            label, idx = labels[i * col + j], i * col + j
            data: np.ndarray = dgroup[idx] if isinstance(dgroup, list) else dgroup[:, idx]
            axe.plot(data); axe.set_title(label, loc = 'center', fontproperties = fontsize)
            axe.ticklabel_format(axis = 'both', style = 'plain')
            axe.set_xlabel('Time(fs)', fontproperties = fontsize); axe.set_ylabel('kcal/mol', fontproperties = fontsize)
            axe.tick_params(axis = 'both', labelsize = fontsize.get_size())
            # if (max_value := np.max(np.abs(data))) > 1.0e3 and np.min(np.abs(data)) > 1e-15:
            #     # order = int(np.log10(max_value))
            #     # axe.set_yticks(np.logspace(-order, order, order))
            #     axe.set_yscale('symlog')
            # axe.set_yscale('symlog')
            # axe.legend()

def extraction(data_file: str, global_file: str):
    properties = np.load(global_file)
    data = np.load(data_file)
    head = properties['properties_head']
    x, v, ppties = data['x'], data['v'], data['properties']
    return x, v, ppties, head

fontsize = FontProperties(size = 34)
row, col = 3, 5

intv = 100
basis = 0.01
start = 1
end = 10000

benchmark = [f'data/benchmark_nve_basis{basis}_intv{intv}/frames{start}_{end}.npz', f'data/benchmark_nve_basis{basis}_intv{intv}/GlobalVariable.npz']
control = [f'data/control_nve_basis{basis}_intv{intv}/frames{start}_{end}.npz', f'data/control_nve_basis{basis}_intv{intv}/GlobalVariable.npz']
taylor = [f'data/taylor_nve_basis{basis}_intv{intv}_iter500/frames{start}_{end}.npz', f'data/taylor_nve_basis{basis}_intv{intv}_iter500/GlobalVariable.npz']

skip = 1
start = 0

# src_x, src_v, src_ppties, head = extraction(*benchmark)
# x, v, ppties = src_x[::skip][start:], src_v[::skip][start:], src_ppties[::skip][start:]

# src_x, src_v, src_ppties, head = extraction(*control)
# x, v, ppties = src_x[start:], src_v[start:], src_control_ppties[start:]

src_x, src_v, src_ppties, head = extraction(*taylor)
x, v, ppties = src_x[start:], src_v[start:], src_ppties[start:]

ntrajs = src_x.shape[0]

# big_Delta_t = big_ntimestep * big_basis_timestep
# taylor_Delta_t = taylor_ntimestep * taylor_basis_timestep

# fig = plt.figure(figsize=[120,30], dpi=500)
fig = plt.figure(figsize=[150, 40], dpi = 200)
axes = fig.subplots(row, col)
draw(axes, row, col, ppties, head, fontsize)

plt.show()
plt.close()

## $$2 \ \ \textit{properties difference}$$

In [ ]:
%set_env OMP_NUM_THREADS=2
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.pyplot import Axes
from matplotlib import font_manager
from matplotlib.font_manager import FontProperties

from concurrent.futures import ThreadPoolExecutor

from typing import List, Dict

basis_properties = ['x', 'v', 'properties']
properties_head = [

    "pe", # total potential energy
    "ke", # kinetic energy
    "etotal", # total energy (pe + ke)
    "evdwl", # van der Waals pairwise energy (includes etail)
    "ecoul", # Coulombic pairwise energy

    "epair", # pairwise energy (evdwl + ecoul + elong)
    "ebond", # bond energy
    "eangle", # angle energy
    "edihed", # dihedral energy
    "eimp", # improper energy
    
    "emol", # molecular energy (ebond + eangle + edihed + eimp)
    "elong", # long-range kspace energy
    "etail", # van der Waals energy long-range tail correction
    "enthalpy", # enthalpy (etotal + press*vol)
    "ecouple", # cumulative energy change due to thermo/baro statting fixes
    "econserve", # pe + ke + ecouple = etotal + ecouple
]

""" Temp Press PotEng KinEng Enthalpy E_vdwl E_coul E_pair E_bond E_angle E_dihed E_long E_tail E_mol Ecouple Econserve TotEng Lx Ly Lz"""
energy_unit = "kcal $\\cdot$ mol$^{-1}$"
press_unit = "ATM"
temperature = "K"
distance_unit = "Angstrom"
time_unit = "fs"
thermo_style_unit = {
    'Temp': f'Kelvin ({temperature})',
    'Press': f'ATMosphere ({press_unit})', 'press': f'ATMosphere ({press_unit})',
    'PotEng': f'energy ({energy_unit})', "pe": f'energy ({energy_unit})',
    'KinEng': f'energy ({energy_unit})', "ke": f'energy ({energy_unit})',
    'Enthalpy': f'energy ({energy_unit})', "enthalpy": f'energy ({energy_unit})',
    'E_vdwl': f'energy ({energy_unit})', "evdwl": f'energy ({energy_unit})',
    'E_coul': f'energy ({energy_unit})', "ecoul": f'energy ({energy_unit})',
    'E_pair': f'energy ({energy_unit})', "epair": f'energy ({energy_unit})',
    'E_bond': f'energy ({energy_unit})', "ebond": f'energy ({energy_unit})',
    'E_angle': f'energy ({energy_unit})', "eangle": f'energy ({energy_unit})',
    'E_dihed': f'energy ({energy_unit})', "edihed": f'energy ({energy_unit})',
                                          "eimp": f'energy ({energy_unit})',
    'E_long': f'energy ({energy_unit})', "elong": f'energy ({energy_unit})',
    'E_tail': f'energy ({energy_unit})', "etail": f'energy ({energy_unit})',
    'E_mol': f'energy ({energy_unit})', "emol": f'energy ({energy_unit})',
    'Ecouple': f'energy ({energy_unit})',  "ecouple": f'energy ({energy_unit})',
    'Econserve': f'energy ({energy_unit})', "econserve": f'energy ({energy_unit})',
    'TotEng': f'energy ({energy_unit})', "etotal": f'energy ({energy_unit})',
    'Lx': f'length ({distance_unit})',  'lx': f'length ({distance_unit})',
    'Ly': f'length ({distance_unit})', 'ly': f'length ({distance_unit})',
    'Lz': f'length ({distance_unit})', 'lz': f'length ({distance_unit})',
}


def draw_difference(
    Canvas: Axes, 
    row: int, col: int, 
    src_dgroup: list | np.ndarray, dst_dgroup: list | np.ndarray, x: np.ndarray, 
    titles: np.ndarray, labels: list, 
    lines_name: list,
    fontsize: FontProperties | None = None,
    skip: int = 1, start = 0,
):
    for i in range(row):
        for j in range(col):
            if i * col + j >= len(titles):
                return
            axe: Axes = Canvas[i, j]
            axe, title, idx = Canvas[i, j], titles[i * col + j], i * col + j
            src_data: np.ndarray = src_dgroup[idx] if isinstance(src_dgroup, list) else src_dgroup[:, idx]
            dst_data: np.ndarray = dst_dgroup[idx] if isinstance(dst_dgroup, list) else dst_dgroup[:, idx]
            # src_mean = np.mean(src_data)
            # dst_mean = np.mean(dst_data)
            dmean = np.mean((src_data + dst_data)/2)
            diff = src_data - dst_data
            axe.plot(x[::skip][start:], dst_data[::skip][start:] - dmean, linewidth = 3); axe.plot(x[::skip][start:], src_data[::skip][start:] - dmean, linewidth = 2)
            axe.plot(x[::skip][start:], diff[::skip][start:], linewidth = 1)
            axe.set_title(f'{title}, MAE of total trajectory = {np.mean(np.abs(diff)):.5}', loc = 'center', fontproperties = fontsize)
            axe.set_xlabel(labels[0], fontproperties = fontsize); axe.set_ylabel(labels[1][title], fontproperties = fontsize)
            axe.tick_params(axis = 'both', labelsize = fontsize.get_size())
            # if (max_value := np.max(np.abs(src_data))) > 1.0e3:
            #     # order = np.log10(max_value, dtype = np.int32)
            #     # axe.set_yticks(np.logspace(-order, order, order + 1))
            #     axe.set_yscale('symlog')
            axe.legend(
                labels = [
                    f"{lines_name[1]}   = y {'+' if dmean >= 0. else '-'} {abs(dmean):.3f}", 
                    f"{lines_name[0]}   = y {'+' if dmean >= 0. else '-'} {abs(dmean):.3f}", 
                    f'error = {lines_name[0]} - {lines_name[1]}'
                ], 
                fontsize = fontsize.get_size(), 
            )

def extraction(folder: str, data_file: str, global_file: str, thermo_file: str):
    properties = np.load(folder + '/' + global_file)
    data = np.load(folder + '/' + data_file)
    thermo_data = np.load(folder + '/' + thermo_file)
    head, thermo_head = properties['properties_head'], properties['thermo_style']
    x, v, ppties = data['x'], data['v'], data['properties']
    return x, v, ppties, head, thermo_head, thermo_data

fontsize = FontProperties(size = 25)
row, col = 4, 4

intv = 100
basis = 0.01
start = 1
end = 10000
save_prefix = "ppties"

benchmark = [f'data/beta_benchmark_nve_basis{basis}_intv{intv}', f'frames{start}_{end}.npz', 'GlobalVariable.npz', 'thermo.npy']; src_name = 'benchmark'
compare = [f'data/beta_control_nve_basis{basis}_intv{intv}', f'frames{start}_{end}.npz', 'GlobalVariable.npz', 'thermo.npy']; compare_name = 'control'
# compare = [f'data/beta_taylor_nve_basis{basis}_intv{intv}_iter500/', f'frames{start}_{end}.npz', 'GlobalVariable.npz', 'thermo.npy']; compare_name = 'taylor'

# read data by multi-thread
# with ThreadPoolExecutor(max_workers = 2) as executor:
#     futures = [executor.submit(extraction, *benchmark), executor.submit(extraction, *taylor)]

#     src_benchmark_x, src_benchmark_v, src_benchmark_ppties, benchmark_head, benchmark_thermo_style, benchmark_thermo_data = futures[0].result()
#     src_taylor_x, src_taylor_v, src_taylor_ppties, taylor_head, taylor_thermo_style, taylor_thermo_data = futures[1].result()

benchmark_x, benchmark_v, benchmark_ppties, benchmark_head, benchmark_thermo_style, benchmark_thermo_data = extraction(*benchmark)
compare_x, compare_v, compare_ppties, compare_head, compare_thermo_style, compare_thermo_data = extraction(*compare)


ntrajs = benchmark_x.shape[0]
x = np.arange(ntrajs)

fig = plt.figure(figsize=[12*col, 7*row], dpi = 200)
axes = fig.subplots(row, col)
draw_difference(axes, row, col, benchmark_ppties, compare_ppties, x, benchmark_head, [f'frames( $\\times$ {basis * intv} fs)', thermo_style_unit], [src_name, compare_name], fontsize, skip = 125)

plt.tight_layout()

# plt.savefig(f"../Image/pdf/benchmark_{compare_name}/{save_prefix}_b{basis}_intv{intv}_{start}_{end}.pdf")
# plt.savefig(f"../Image/png/benchmark_{compare_name}/{save_prefix}_b{basis}_intv{intv}_{start}_{end}.png")

plt.savefig(f"../Image_beta/pdf/benchmark_{compare_name}/{save_prefix}_b{basis}_intv{intv}_{start}_{end}.pdf")
plt.savefig(f"../Image_beta/png/benchmark_{compare_name}/{save_prefix}_b{basis}_intv{intv}_{start}_{end}.png")

plt.show()
plt.close()

## $$3 \ \ \textit{coordinate and velocity difference}$$

In [ ]:
%matplotlib inline
# %matplotlib Qt5
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.pyplot import Axes
from matplotlib import font_manager
from matplotlib.font_manager import FontProperties

from einops import rearrange

np.set_printoptions(threshold = np.inf)

def draw_difference(axe: Axes, data: np.ndarray, frame: np.ndarray, title: str, labels: list[str], fontsize: FontProperties | None = None, skip = 1, start = 0):

    axe.plot(frame[::skip][start:], data[::skip][start:], linewidth = 10)
    data_mean = np.mean(np.abs(data))
    axe.set_title(f'{title}, range from [{frame[0]}, {frame[-1] + 1})', loc = 'center', fontproperties = fontsize)
    axe.axhline(y = data_mean, linewidth = 6, color = 'r', linestyle = '--')
    axe.set_xlabel(labels[1], fontproperties = fontsize); axe.set_ylabel(labels[2], fontproperties = fontsize)
    axe.tick_params(axis = 'both', labelsize = fontsize.get_size())
    # axe.set_yscale('symlog')
    exp_idx = int(np.floor(np.log10(data_mean)))
    axe.legend([
        labels[0], 
        f"MAE of total: {data_mean * 10 ** (-exp_idx):.3f} $\\times$ 10$^{{{exp_idx}}}$"
        ], fontsize = fontsize.get_size(), loc = 'lower right')

def extraction(folder: str, data_file: str, global_file: str, *, id_filter: str = None, type_filter: str = None):
    properties = np.load(folder + '/' + global_file)
    data: dict = np.load(folder + '/' + data_file)

    mass, id, atom_type = data.get('mass', None), data.get('id', None), data.get('atom_type', None)
    
    delta_t = properties['basis_timestep'] * properties['ntimestep']

    x, v = data['x'], data['v']


    if id_filter is not None and id is not None:
        st, en = id_filter.split(":")
        mask = np.logical_and(id >= int(st), id <= int(en))
        x, v, atom_type, id = x[:, mask], v[:, mask], atom_type[mask], id[mask]

    if type_filter is not None and atom_type is not None:
        st, en = type_filter.split(":")
        mask = np.logical_and(atom_type >= int(st), atom_type <= int(en))
        x, v, atom_type, id = x[:, mask], v[:, mask], atom_type[mask], id[mask]

    # attn check for pbc
    boundary = properties['boundary']

    blo, bhi = boundary
    blen = bhi - blo

    x = np.where(x < blo, x + blen, x)
    x = np.where(x < bhi, x, x - blen)

    return x, v, delta_t, boundary

fontsize = FontProperties(size = 25)
row, col = 1, 2

intv = 320
basis = 0.1
start = 1
end = 10000
# group = "3:18"
group = None

benchmark = [f'data/benchmark_nve_basis{basis}_intv{intv}', f'frames{start}_{end}.npz', 'GlobalVariable.npz']; src_name = 'benchmark'
# compare = [f'data/control_nve_basis{basis}_intv{intv}', f'frames{start}_{end}.npz', 'GlobalVariable.npz']; compare_name = 'control'
compare = [f'data/taylor_nve_basis{basis}_intv{intv}_iter500/', f'frames{start}_{end}.npz', 'GlobalVariable.npz']; compare_name = 'taylor'

src_benchmark_x, src_benchmark_v, benchmark_delta_t, benchmark_boudary = extraction(*benchmark, type_filter = group)
src_compare_x, src_compare_v, compare_delta_t, compare_boundary = extraction(*compare, type_filter = group)

frame = np.arange(start, end + 1)

start = 0

benchmark_x, benchmark_v = src_benchmark_x[start:], src_benchmark_v[start:]
compare_x, compare_v = src_compare_x[start:], src_compare_v[start:]
frame = frame[start:]

ntrajs = src_benchmark_x.shape[0]

assert np.sum(np.abs(compare_boundary - benchmark_boudary)) < 1e-10
boundary = benchmark_boudary

xdiff = benchmark_x - compare_x
vdiff = benchmark_v - compare_v

# pbc
blo, bhi = boundary
blen = rearrange(bhi - blo, "ndims -> 1 1 ndims")
xdiff = np.where(xdiff < blen * 0.5, xdiff, xdiff - blen)
xdiff = np.where(xdiff > -blen * 0.5, xdiff, xdiff + blen)

xdiff = np.mean(np.reshape(np.abs(xdiff), (xdiff.shape[0], -1)), axis = -1)
vdiff = np.mean(np.reshape(np.abs(vdiff), (vdiff.shape[0], -1)), axis = -1)

# xdiff = np.mean(np.abs(benchmark_x - compare_x), axis = (-1, -2))
# vdiff = np.mean(np.abs(benchmark_v - compare_v), axis = (-1, -2))

# xdiff = np.abs(benchmark_x - compare_x)[:1000]
# xdiff = np.where(1e-2 < xdiff < 10., True, False)
# mask = np.logical_and(xdiff > 1e-2, xdiff < 10.0)
# count = np.count_nonzero(mask)
# print(count)

fig = plt.figure(figsize=[12*col, 7*row], dpi = 200)
coords_axes, velocity_axes = fig.subplots(row, col)


draw_difference(coords_axes, xdiff, frame, 'coordinates difference', [f'{src_name} - {compare_name}', f'Frames ($\\times${benchmark_delta_t} fs)', f'Distance(Angstrom)'], fontsize, skip = 25)
draw_difference(velocity_axes, vdiff, frame, 'velocities difference', [f'{src_name} - {compare_name}', f'Frames, ($\\times${benchmark_delta_t} fs)', f'Distance/time (Angstrom/fs)'], fontsize, skip = 25)

plt.tight_layout()

plt.show()
plt.close()

## $$4 \ \ \textit{Radius of gyration}$$

this Equation is described as follows: 
$$
R_g = (\frac{\sum_i ||r_i||^2 m_i}{\sum_i m_i})^\frac{1}{2}
$$

where $m_i$ is the mass of atom $i$ and $r_i$ the position of atom $i$ with respect to the center of mass of the molecule. It is especially useful to characterize polymer solutions and proteins. 

Center of Mass of the molecule: 
$$
M_{cm} = \frac{\int \rho(r)rdr}{\int \rho(r)dr}
$$

In [ ]:
"""
    Solution: put COM of the protein in all frames to the center of the box. and then compute the Radius of gyration.
"""
%matplotlib inline
import numpy as np
from numpy import ndarray as nparr
from matplotlib import pyplot as plt
from matplotlib.pyplot import Axes
from matplotlib import font_manager
from matplotlib.font_manager import FontProperties

from einops import repeat, rearrange

np.set_printoptions(threshold = np.inf)

def extraction(folder: str, data_file: str, global_file: str, filter: str = None):
    id_filter = None
    type_filter = None
    
    ftype, group = filter.split(" ")
    match ftype:
        case 'id':
            id_filter = group
        case 'type':
            type_filter = group
        case _:
            raise ValueError

    properties = np.load(folder + '/' + global_file)
    data: dict = np.load(folder + '/' + data_file)

    mass, id, atom_type = data.get('mass', None), data.get('id', None), data.get('atom_type', None)
    
    delta_t = properties['basis_timestep'] * properties['ntimestep']

    x, v = data['x'], data['v']

    if id_filter is not None and id is not None:
        st, en = id_filter.split(":")
        mask = np.logical_and(id >= int(st), id <= int(en))
        x, v, atom_type, id = x[:, mask], v[:, mask], atom_type[mask], id[mask]
    
    if type_filter is not None and atom_type is not None:
        st, en = type_filter.split(":")
        mask = np.logical_and(atom_type >= int(st), atom_type <= int(en))
        x, v, atom_type, id = x[:, mask], v[:, mask], atom_type[mask], id[mask]

    # attn check for pbc
    boundary = properties['boundary']

    blo, bhi = boundary
    blen = bhi - blo

    x = np.where(x < blo, x + blen, x)
    x = np.where(x < bhi, x, x - blen)

    return x, v, delta_t, mass[atom_type], atom_type, id, boundary

def compute_COM(x: nparr, mass: nparr):

    ntrajs, natoms, ndims = x.shape

    sum_of_mass = np.sum(mass)

    mass = repeat(mass, "natoms -> ntrajs natoms ndims", ntrajs = ntrajs, ndims = ndims)

    com = np.sum(mass * x, axis = 1) / sum_of_mass

    return com
    
def compute_RG(x: nparr, mass: nparr):

    ntrajs, natoms, ndims = x.shape

    com = compute_COM(x, mass)

    com = repeat(com, "ntrajs ndims -> ntrajs natoms ndims", natoms = natoms)

    sum_of_mass = np.sum(mass)

    mass = repeat(mass, "natoms -> ntrajs natoms", ntrajs = ntrajs)

    r_sq_m = np.sum((x - com) ** 2, axis = -1) * mass

    Rg_sq = np.sum(r_sq_m, axis = -1) / sum_of_mass

    Rg = np.sqrt(Rg_sq)

    return Rg

def adjust_RG(coords: nparr, boundary: nparr, mass: nparr):

    blo, bhi = np.split(boundary, [1], axis = 0)
    blen = bhi - blo
    
    box_center = (bhi + blo) / 2

    com = compute_COM(coords, mass)

    offset = rearrange(com - box_center, "ntrajs ndims -> ntrajs 1 ndims")
    
    coords = coords - offset
    
    # 移动后再执行一次 pbc
    coords = np.where(coords < blo, coords + blen, coords)
    coords = np.where(coords < bhi, coords, coords - blen)

    return coords
    

fontsize = FontProperties(size = 34)

intv = 100
basis = 0.01
start = 1
end = 10000
group = "id 1:10"
save_prefix = "Rg"

benchmark = [
    f'data/benchmark_nve_basis{basis}_intv{intv}/',
    f'frames{start}_{end}.npz', 
    f'GlobalVariable.npz',
]
# compare = [
#     f'data/control_nve_basis{basis}_intv{intv}/',
#     f'frames{start}_{end}.npz', 
#     f'GlobalVariable.npz',
# ]; compare_name = 'control'
compare = [
    f'data/taylor_nve_basis{basis}_intv{intv}_iter500/',
    f'frames{start}_{end}.npz', 
    f'GlobalVariable.npz',
]; compare_name = 'taylor'

# benchmark = [
#     f'data/beta_benchmark_nve_basis{basis}_intv{intv}/',
#     f'frames{start}_{end}.npz', 
#     f'GlobalVariable.npz',
# ]
# compare = [
#     f'data/beta_control_nve_basis{basis}_intv{intv}/',
#     f'frames{start}_{end}.npz', 
#     f'GlobalVariable.npz',
# ]; compare_name = 'control'
# compare = [
#     f'data/beta_taylor_nve_basis{basis}_intv{intv}_iter500/',
#     f'frames{start}_{end}.npz', 
#     f'GlobalVariable.npz',
# ]; compare_name = 'taylor'


# mass shape: (natoms,), id shape: (natoms,),  x shape: (ntrajs, natoms, 3), v shape: (ntrajs, natoms, 3)
benchmark_x, benchmark_v, benchmark_delta_t, benchmark_mass, benchmark_atype, benchmark_id, benchmark_boundary = extraction(*benchmark, type_filter = group)
compare_x, compare_v, compare_delta_t, compare_mass, compare_atype, compare_id, compare_boundary = extraction(*compare, type_filter = group)

mol_idx = 4

benchmark_x, benchmark_v, benchmark_mass, benchmark_atype, benchmark_id = benchmark_x[:,mol_idx * 16:(mol_idx + 1) *16], benchmark_v[:, mol_idx * 16:(mol_idx + 1) *16], benchmark_mass[mol_idx * 16:(mol_idx + 1) *16], benchmark_atype[mol_idx * 16:(mol_idx + 1) *16], benchmark_id[mol_idx * 16:(mol_idx + 1) *16]
compare_x, compare_v, compare_mass, compare_atype, compare_id = compare_x[:,mol_idx * 16:(mol_idx + 1) *16], compare_v[:, mol_idx * 16:(mol_idx + 1) *16], compare_mass[mol_idx * 16:(mol_idx + 1) *16], compare_atype[mol_idx * 16:(mol_idx + 1) *16], compare_id[mol_idx * 16:(mol_idx + 1) *16]


adjust_benchmark_x = adjust_RG(benchmark_x, benchmark_boundary, benchmark_mass)
adjust_compare_x = adjust_RG(compare_x, compare_boundary, compare_mass)

frame = np.arange(start, end + 1)

benchmark_Rg = compute_RG(adjust_benchmark_x, benchmark_mass)
compare_Rg = compute_RG(adjust_compare_x, compare_mass)
Abe = np.abs(benchmark_Rg - compare_Rg)

### $$ 4.1 \ \ \textit{Visualization of Rg}$$

In [ ]:
skip = 100
begin = 0

fig = plt.figure(figsize=[60, 30], dpi = 200)
Rg_axes, difference_axes = fig.subplots(2, 1)



Rg_axes.plot(frame[::skip][begin:], benchmark_Rg[::skip][begin:], linewidth = 8)
Rg_axes.plot(frame[::skip][begin:], compare_Rg[::skip][begin:], linewidth = 5)
Rg_axes.set_title(f'Radius of gyration of {mol_idx}-Indole, range [{start},{end + 1})', fontsize = 60)
Rg_axes.set_ylabel('Distance(Angstrom)', fontsize = 40)
Rg_axes.set_xlabel(f'frames($\\times${intv * basis} fs $\\cdot$ frame$^{{-1}}$)', fontsize = 40)
Rg_axes.tick_params(axis = 'both', labelsize = 40)
Rg_axes.legend([f'benchmark {basis}fs', f'{compare_name} {intv*basis}fs'], fontsize = 51, loc = 'lower right')


mae = np.mean(Abe)
exp = int(np.floor(np.log10(mae)))
coe = round(mae * (10 ** (-exp)), abs(exp))

difference_axes.plot(frame[::skip][begin:], Abe[::skip][begin:], linewidth = 6)
difference_axes.axhline(y = mae, color = 'r', linestyle = '--', linewidth = 8)
difference_axes.set_title(f'Absolute Error of each Rg', fontsize = 60)
difference_axes.set_ylabel('Distance(Angstrom)', fontsize = 40)
difference_axes.set_xlabel(f'frames($\\times${intv*basis} fs $\\cdot$ frame$^{{-1}}$)', fontsize = 40)
difference_axes.tick_params(axis = 'both', labelsize = 40)
# difference_axes.set_yscale('symlog')
difference_axes.legend(['Mean Absolute Error', f'Average {coe} $\\times$ 10$^{{{exp}}}$'], fontsize = 51, loc = 'upper left')

plt.tight_layout()

# plt.savefig(f"../Image/pdf/benchmark_{compare_name}/{save_prefix}_b{basis}_intv{intv}_{start}_{end}.pdf")
# plt.savefig(f"../Image/png/benchmark_{compare_name}/{save_prefix}_b{basis}_intv{intv}_{start}_{end}.png")

plt.savefig(f"../Image_beta/pdf/benchmark_{compare_name}/{save_prefix}_b{basis}_intv{intv}_{start}_{end}.pdf")
plt.savefig(f"../Image_beta/png/benchmark_{compare_name}/{save_prefix}_b{basis}_intv{intv}_{start}_{end}.png")

plt.show()
plt.close()

## $$5 \ \ \textit{Root Mean Square Deviation}$$


this equation is described as follows:

$$
RMSD(t_1,t_2) ~=~ \left[\frac{1}{M} \sum_{i=1}^N m_i \|{\bf r}_i(t_1)-{\bf r}_i(t_2)\|^2 \right]^{\frac{1}{2}}
$$

where $M = \sum_{i=1}^N m_i$ and ${\bf r}_i(t)$ is the position of atom $i$ at time $t$. Note that fitting does not have to use the same atoms as the calculation of the $RMSD$; e.g. a protein is usually fitted on the backbone atoms $(N, C_{\alpha}, C)$, but the $RMSD$ can be computed of the backbone or of the whole protein.

In [ ]:
"""
    Solution: put COM of the protein in the first frame to the center of the box. 
"""

%matplotlib inline
import numpy as np
from numpy import ndarray as nparr
from matplotlib import pyplot as plt
from matplotlib.pyplot import Axes
from matplotlib import font_manager
from matplotlib.font_manager import FontProperties

from einops import repeat, rearrange

np.set_printoptions(threshold = np.inf)

def extraction(folder: str, data_file: str, global_file: str, filter: str = None):
    id_filter = None
    type_filter = None
    
    ftype, group = filter.split(" ")
    match ftype:
        case 'id':
            id_filter = group
        case 'type':
            type_filter = group
        case _:
            raise ValueError

    properties = np.load(folder + '/' + global_file)
    data: dict = np.load(folder + '/' + data_file)

    mass, id, atom_type = data.get('mass', None), data.get('id', None), data.get('atom_type', None)
    
    delta_t = properties['basis_timestep'] * properties['ntimestep']

    x, v = data['x'], data['v']

    if id_filter is not None and id is not None:
        st, en = id_filter.split(":")
        mask = np.logical_and(id >= int(st), id <= int(en))
        x, v, atom_type, id = x[:, mask], v[:, mask], atom_type[mask], id[mask]
    
    if type_filter is not None and atom_type is not None:
        st, en = type_filter.split(":")
        mask = np.logical_and(atom_type >= int(st), atom_type <= int(en))
        x, v, atom_type, id = x[:, mask], v[:, mask], atom_type[mask], id[mask]

    # attn check for pbc
    boundary = properties['boundary']

    blo, bhi = boundary
    blen = bhi - blo

    x = np.where(x < blo, x + blen, x)
    x = np.where(x < bhi, x, x - blen)

    return x, v, delta_t, mass[atom_type], atom_type, id, boundary


def adjust_RMSD(coords: nparr, boundary: nparr, mass: nparr):

    # ntrajs, natoms, ndims = coords.shape

    # sum_of_mass = np.sum(mass)

    # mass = rearrange(mass, "natoms -> natoms 1")

    # fcom = np.sum(mass * coords[0], axis = 0, keepdims = True) / sum_of_mass

    blo, bhi = np.split(boundary, [1], axis = 0)
    blen = bhi - blo
    
    # box_center = (bhi + blo) / 2

    # offset = rearrange(fcom - box_center, "1 ndims -> 1 1 ndims")
    
    # coords = coords - offset
    
    # # 移动后再执行一次 pbc
    # coords = np.where(coords < blo, coords + blen, coords)
    # coords = np.where(coords < bhi, coords, coords - blen)

    distance = np.diff(coords, prepend = coords[0:1], axis = 0)
    check = np.zeros_like(distance)
    check = np.where(distance >= blen / 2, check - 1, check)
    check = np.where(distance <= -blen / 2, check + 1, check)
    check = np.cumsum(check, axis = 0)

    coords = coords + check * blen 

    return coords

def compute_RMSD(trajs: nparr, mass: nparr, t2: int = 0):
    
    ntrajs, natoms, ndims = trajs.shape
    
    rt2 = trajs[t2]

    sum_of_mass = np.sum(mass)

    mass = rearrange(mass, "natoms -> 1 natoms")

    diff = np.sum((trajs - rt2) ** 2, axis = -1) * mass / sum_of_mass

    diff = np.sum(diff, axis = -1) ** 0.5

    return diff


fontsize = FontProperties(size = 34)

intv = 400
basis = 0.01

g = 0
mol_idx = 3
start = 1 + g * 10000
end = 10000 + g * 10000

group = "type 3:18"
save_prefix = "rmsd"

# benchmark = [
#     f'data/benchmark_nve_basis{basis}_intv{intv}/',
#     f'frames{start}_{end}.npz',
#     f'GlobalVariable.npz',
# ]
# # compare = [
# #     f'data/control_nve_basis{basis}_intv{intv}/',
# #     f'frames{start}_{end}.npz', 
# #     f'GlobalVariable.npz',
# # ]; compare_name = 'control'
# compare = [
#     f'data/taylor_nve_basis{basis}_intv{intv}_iter500/',
#     f'frames{start}_{end}.npz', 
#     f'GlobalVariable.npz',
# ]; compare_name = 'taylor'

benchmark = [
    f'data/beta_benchmark_nve_basis{basis}_intv{intv}/',
    f'frames{start}_{end}.npz', 
    f'GlobalVariable.npz',
]
# compare = [
#     f'data/beta_control_nve_basis{basis}_intv{intv}/',
#     f'frames{start}_{end}.npz', 
#     f'GlobalVariable.npz',
# ]; compare_name = 'control'
compare = [
    f'data/beta_taylor_nve_basis{basis}_intv{intv}_iter500/',
    f'frames{start}_{end}.npz', 
    f'GlobalVariable.npz',
]; compare_name = 'taylor'

# mass shape: (natoms,), id shape: (natoms,),  x shape: (ntrajs, natoms, 3), v shape: (ntrajs, natoms, 3)
benchmark_x, benchmark_v, benchmark_delta_t, benchmark_mass, benchmark_atype, benchmark_id, benchmark_boundary = extraction(*benchmark, filter = group)
compare_x, compare_v, compare_delta_t, compare_mass, compare_atype, compare_id, compare_boundary = extraction(*compare, filter = group)

frame = np.arange(start, end + 1)

id_group = np.argsort(benchmark_id)

benchmark_x, benchmark_v, benchmark_mass, benchmark_atype, benchmark_id = benchmark_x[:, id_group], benchmark_v[:, id_group], benchmark_mass[id_group], benchmark_atype[id_group], benchmark_id[id_group]
compare_x, compare_v, compare_mass, compare_atype, compare_id = compare_x[:, id_group], compare_v[:, id_group], compare_mass[id_group], compare_atype[id_group], compare_id[id_group]

benchmark_x, benchmark_v, benchmark_mass, benchmark_atype, benchmark_id = benchmark_x[:,mol_idx * 16:(mol_idx + 1) *16], benchmark_v[:, mol_idx * 16:(mol_idx + 1) *16], benchmark_mass[mol_idx * 16:(mol_idx + 1) *16], benchmark_atype[mol_idx * 16:(mol_idx + 1) *16], benchmark_id[mol_idx * 16:(mol_idx + 1) *16]
compare_x, compare_v, compare_mass, compare_atype, compare_id = compare_x[:,mol_idx * 16:(mol_idx + 1) *16], compare_v[:, mol_idx * 16:(mol_idx + 1) *16], compare_mass[mol_idx * 16:(mol_idx + 1) *16], compare_atype[mol_idx * 16:(mol_idx + 1) *16], compare_id[mol_idx * 16:(mol_idx + 1) *16]

print(benchmark_id, benchmark_atype)
print(compare_id, compare_atype)

adjust_benchmark_x = adjust_RMSD(benchmark_x, benchmark_boundary, benchmark_mass)
adjust_compare_x = adjust_RMSD(compare_x, compare_boundary, compare_mass)

benchmark_RMSD = compute_RMSD(adjust_benchmark_x, benchmark_mass, 0)
compare_RMSD = compute_RMSD(adjust_compare_x, compare_mass, 0)

Abe = np.abs(benchmark_RMSD - compare_RMSD)

### $$ 5.1 \ \ \textit{Visualization of RMSD}$$

In [ ]:
skip = 100
begin = 0


fig = plt.figure(figsize=[25, 15], dpi = 200)
Rg_axes, difference_axes = fig.subplots(2, 1)

Rg_axes.plot(frame, benchmark_RMSD, linewidth = 6)
Rg_axes.plot(frame, compare_RMSD, linewidth = 3)
Rg_axes.set_title(f'Root Mean Square Deviation of {mol_idx}-Indole, range from [{start},{end + 1})', fontsize = fontsize.get_size())
Rg_axes.set_ylabel('Distance(Angstrom)', fontsize = fontsize.get_size())
Rg_axes.set_xlabel(f'frames($\\times${intv * basis} fs $\\cdot$ frame$^{{-1}}$)', fontsize = fontsize.get_size())
Rg_axes.tick_params(axis = 'both', labelsize = fontsize.get_size())
Rg_axes.legend([f'benchmark {basis}fs', f'{compare_name} {intv*basis}fs'], fontsize = fontsize.get_size(), loc = 'lower right')

mae = np.mean(Abe)
exp = int(np.floor(np.log10(mae)))
coe = round(mae * (10 ** (-exp)), abs(exp) + 1)


difference_axes.plot(frame, Abe, linewidth = 4)
difference_axes.axhline(y = mae, color = 'g', linestyle = '--', linewidth = 6)
difference_axes.set_title(f'Absolute Error of each RMSD', fontsize = fontsize.get_size())
difference_axes.set_ylabel('Distance(Angstrom)', fontsize = fontsize.get_size())
difference_axes.set_xlabel(f'frames($\\times${intv * basis} fs $\\cdot$ frame$^{{-1}}$)', fontsize = fontsize.get_size())
difference_axes.tick_params(axis = 'both', labelsize = fontsize.get_size())
# difference_axes.set_yscale('symlog')
difference_axes.legend(['Mean Absolute Error', f'average {coe} $\\times$ 10$^{{{exp}}}$'], fontsize = fontsize.get_size(), loc = 'upper left')

plt.tight_layout()

# plt.savefig(f"../Image/pdf/benchmark_{compare_name}/{save_prefix}_indole-{mol_idx}_b{basis}_intv{intv}_{start}_{end}.pdf")
# plt.savefig(f"../Image/png/benchmark_{compare_name}/{save_prefix}_indole-{mol_idx}_b{basis}_intv{intv}_{start}_{end}.png")

plt.savefig(f"../Image_beta/pdf/benchmark_{compare_name}/{save_prefix}_indole-{mol_idx}_b{basis}_intv{intv}_{start}_{end}.pdf")
plt.savefig(f"../Image_beta/png/benchmark_{compare_name}/{save_prefix}_indole-{mol_idx}_b{basis}_intv{intv}_{start}_{end}.png")

plt.show()
plt.close()

## $$6 \ \ \textit{Mean Square Displacement}$$


this equation is described as follows:

$$
MSD(t_1,t_2) ~=~ \frac{1}{N} \sum_{i=1}^N \|{\bf r}_i(t_1)-{\bf r}_i(t_2)\|^2 
$$

where ${\bf r}_i(t)$ is the position of atom $i$ at time $t$. Note that fitting does not have to use the same atoms as the calculation of the $MSD$; e.g. a protein is usually fitted on the backbone atoms $(N, C_{\alpha}, C)$, but the $MSD$ can be computed of the backbone or of the whole protein.

In [ ]:
"""
    Solution: put center of the protein in the first frame to the center of the box. 
"""

%matplotlib inline
import numpy as np
from numpy import ndarray as nparr
from matplotlib import pyplot as plt
from matplotlib.pyplot import Axes
from matplotlib import font_manager
from matplotlib.font_manager import FontProperties

from einops import repeat, rearrange
from typing import Literal

np.set_printoptions(threshold = np.inf)

def extraction(folder: str, data_file: str, global_file: str, filter: str = None):
    id_filter = None
    type_filter = None
    
    ftype, group = filter.split(" ")
    match ftype:
        case 'id':
            id_filter = group
        case 'type':
            type_filter = group
        case _:
            raise ValueError

    properties = np.load(folder + '/' + global_file)
    data: dict = np.load(folder + '/' + data_file)

    mass, id, atom_type = data.get('mass', None), data.get('id', None), data.get('atom_type', None)
    
    delta_t = properties['basis_timestep'] * properties['ntimestep']

    x, v = data['x'], data['v']

    if id_filter is not None and id is not None:
        st, en = id_filter.split(":")
        mask = np.logical_and(id >= int(st), id <= int(en))
        x, v, atom_type, id = x[:, mask], v[:, mask], atom_type[mask], id[mask]
    
    if type_filter is not None and atom_type is not None:
        st, en = type_filter.split(":")
        mask = np.logical_and(atom_type >= int(st), atom_type <= int(en))
        x, v, atom_type, id = x[:, mask], v[:, mask], atom_type[mask], id[mask]

    # attn check for pbc
    boundary = properties['boundary']

    blo, bhi = boundary
    blen = bhi - blo

    x = np.where(x < blo, x + blen, x)
    x = np.where(x < bhi, x, x - blen)

    return x, v, delta_t, mass[atom_type], atom_type, id, boundary


def adjust_MSD(coords: nparr, boundary: nparr):

    ntrajs, natoms, ndims = coords.shape

    blo, bhi = np.split(boundary, [1], axis = 0)
    blen = bhi - blo

    # distance = coords[1:] - coords[:-1]
    # print(coords)
    distance = np.diff(coords, prepend = coords[0:1], axis = 0)
    check = np.zeros_like(distance)
    check = np.where(distance >= blen / 2, check - 1, check)
    check = np.where(distance <= -blen / 2, check + 1, check)
    check = np.cumsum(check, axis = 0)

    coords = coords + check * blen 

    return coords

def compute_MSD(trajs: nparr, t2: int = 0):
    
    ntrajs, natoms, ndims = trajs.shape
    
    rt2 = trajs[t2]

    diff = np.sum((trajs - rt2) ** 2, axis = -1)

    diff = np.mean(diff, axis = -1)

    return diff


fontsize = FontProperties(size = 34)

intv = 400
basis = 0.01

g = 0
mol_idx = 3
start = 1 + g * 10000
end = 10000 + g * 10000

group = "type 3:18"
save_prefix: Literal['dxv', 'ppties', 'msd'] = "msd"

benchmark = [
    f'data/benchmark_nve_basis{basis}_intv{intv}/',
    f'frames{start}_{end}.npz', 
    f'GlobalVariable.npz',
]
# compare = [
#     f'data/control_nve_basis{basis}_intv{intv}/',
#     f'frames{start}_{end}.npz', 
#     f'GlobalVariable.npz',
# ]; compare_name = 'control'
compare = [
    f'data/taylor_nve_basis{basis}_intv{intv}_iter500/',
    f'frames{start}_{end}.npz', 
    f'GlobalVariable.npz',
]; compare_name = 'taylor'

# benchmark = [
#     f'data/beta_benchmark_nve_basis{basis}_intv{intv}/',
#     f'frames{start}_{end}.npz', 
#     f'GlobalVariable.npz',
# ]
# compare = [
#     f'data/beta_control_nve_basis{basis}_intv{intv}/',
#     f'frames{start}_{end}.npz', 
#     f'GlobalVariable.npz',
# ]; compare_name = 'control'
# # compare = [
# #     f'data/beta_taylor_nve_basis{basis}_intv{intv}_iter500/',
# #     f'frames{start}_{end}.npz', 
# #     f'GlobalVariable.npz',
# # ]; compare_name = 'taylor'

# mass shape: (natoms,), id shape: (natoms,),  x shape: (ntrajs, natoms, 3), v shape: (ntrajs, natoms, 3)
benchmark_x, benchmark_v, benchmark_delta_t, benchmark_mass, benchmark_atype, benchmark_id, benchmark_boundary = extraction(*benchmark, filter = group)
compare_x, compare_v, compare_delta_t, compare_mass, compare_atype, compare_id, compare_boundary = extraction(*compare, filter = group)

frame = np.arange(start, end + 1)

id_group = np.argsort(benchmark_id)

benchmark_x, benchmark_v, benchmark_mass, benchmark_atype, benchmark_id = benchmark_x[:, id_group], benchmark_v[:, id_group], benchmark_mass[id_group], benchmark_atype[id_group], benchmark_id[id_group]
compare_x, compare_v, compare_mass, compare_atype, compare_id = compare_x[:, id_group], compare_v[:, id_group], compare_mass[id_group], compare_atype[id_group], compare_id[id_group]

benchmark_x, benchmark_v, benchmark_mass, benchmark_atype, benchmark_id = benchmark_x[:,mol_idx * 16:(mol_idx + 1) *16], benchmark_v[:, mol_idx * 16:(mol_idx + 1) *16], benchmark_mass[mol_idx * 16:(mol_idx + 1) *16], benchmark_atype[mol_idx * 16:(mol_idx + 1) *16], benchmark_id[mol_idx * 16:(mol_idx + 1) *16]
compare_x, compare_v, compare_mass, compare_atype, compare_id = compare_x[:,mol_idx * 16:(mol_idx + 1) *16], compare_v[:, mol_idx * 16:(mol_idx + 1) *16], compare_mass[mol_idx * 16:(mol_idx + 1) *16], compare_atype[mol_idx * 16:(mol_idx + 1) *16], compare_id[mol_idx * 16:(mol_idx + 1) *16]

print(benchmark_id, benchmark_atype)
print(compare_id, compare_atype)

adjust_benchmark_x = adjust_MSD(benchmark_x, benchmark_boundary)
adjust_compare_x = adjust_MSD(compare_x, compare_boundary)

benchmark_MSD = compute_MSD(adjust_benchmark_x, 0)
compare_MSD = compute_MSD(adjust_compare_x, 0)

Abe = np.abs(benchmark_MSD - compare_MSD)

### $$ 6.1 \ \ \textit{Visualization of MSD}$$

In [ ]:
skip = 100
begin = 0


fig = plt.figure(figsize=[25, 15], dpi = 200)
Rg_axes, difference_axes = fig.subplots(2, 1)

Rg_axes.plot(frame, benchmark_MSD, linewidth = 6)
Rg_axes.plot(frame, compare_MSD, linewidth = 3)
Rg_axes.set_title(f'Mean Square Displacement of {mol_idx}-Indole, range from [{start},{end + 1})', fontsize = fontsize.get_size())
Rg_axes.set_ylabel('Distance(Angstrom$^2$)', fontsize = fontsize.get_size())
Rg_axes.set_xlabel(f'frames($\\times${intv * basis} fs $\\cdot$ frame$^{{-1}}$)', fontsize = fontsize.get_size())
Rg_axes.tick_params(axis = 'both', labelsize = fontsize.get_size())
Rg_axes.legend([f'benchmark {basis}fs', f'{compare_name} {intv*basis}fs'], fontsize = fontsize.get_size())
# Rg_axes.set_yscale('log')

mae = np.mean(Abe)
exp = int(np.floor(np.log10(mae)))
coe = round(mae * (10 ** (-exp)), abs(exp) + 1)


difference_axes.plot(frame, Abe, linewidth = 4)
difference_axes.axhline(y = mae, color = 'g', linestyle = '--', linewidth = 6)
difference_axes.set_title(f'Absolute Error of each MSD', fontsize = fontsize.get_size())
difference_axes.set_ylabel('Distance(Angstrom$^2$)', fontsize = fontsize.get_size())
difference_axes.set_xlabel(f'frames($\\times${intv * basis} fs $\\cdot$ frame$^{{-1}}$)', fontsize = fontsize.get_size())
difference_axes.tick_params(axis = 'both', labelsize = fontsize.get_size())
# difference_axes.set_yscale('symlog')
difference_axes.legend(['Mean Absolute Error', f'average {coe} $\\times$ 10$^{{{exp}}}$'], fontsize = fontsize.get_size(), loc = 'upper left')

plt.tight_layout()

plt.savefig(f"../Image/pdf/benchmark_{compare_name}/{save_prefix}_indole-{mol_idx}_b{basis}_intv{intv}_{start}_{end}.pdf")
plt.savefig(f"../Image/png/benchmark_{compare_name}/{save_prefix}_indole-{mol_idx}_b{basis}_intv{intv}_{start}_{end}.png")

# plt.savefig(f"../Image_beta/pdf/benchmark_{compare_name}/{save_prefix}_indole-{mol_idx}_b{basis}_intv{intv}_{start}_{end}.pdf")
# plt.savefig(f"../Image_beta/png/benchmark_{compare_name}/{save_prefix}_indole-{mol_idx}_b{basis}_intv{intv}_{start}_{end}.png")

plt.show()
plt.close()

In [ ]:
# import numpy as np
# from tqdm import tqdm

# intv = 300
# basis = 0.01
# start = 1
# end = 10000
# mode = 'taylor'


# path = f'data/beta_{mode}_nve_basis{basis}_intv{intv}/' if mode != 'taylor' else f'data/beta_{mode}_nve_basis{basis}_intv{intv}_iter500/'
# pathlist = [x for x in path.split('/') if x != '']
# name = pathlist[-1]
# print(name, path)
# global_file = 'GlobalVariable.npz'
# data_file = f'frames{start}_{end}.npz'

# properties: dict = np.load(path + global_file)
# data: dict = np.load(path + '/' + data_file)


# coord, velocity, atype, aid = data['x'], data['v'], data.get('atom_type', None), data.get('id', None)
# boundary = properties['boundary']
# blo, bhi = boundary

# sort_id = np.argsort(aid)

# coord, velocity, atype, aid = coord[:, sort_id], velocity[:, sort_id], atype[sort_id], aid[sort_id]

# mol_idx = np.where(atype > 2, 2, 444)

# q = np.zeros_like(atype, dtype = np.float16)
# q[atype == 1] = -0.55
# q[atype == 2] = 1.1
# q[atype == 3] = 0.17470000
# q[atype == 4] = -0.22970000
# q[atype == 5] = -0.09740000
# q[atype == 6] = 0.01730000
# q[atype == 7] = 0.19270000
# q[atype == 8] = -0.64230000
# q[atype == 9] = 0.44150000
# q[atype == 10] = 0.12020000
# q[atype == 11] = -0.09210000
# q[atype == 12] = 0.14840000
# q[atype == 13] = -0.16820000
# q[atype == 14] = 0.14340000
# q[atype == 15] = -0.18250000
# q[atype == 16] = 0.14710000
# q[atype == 17] = -0.12110000
# q[atype == 18] = 0.14800000

# # with open(f'{path}/{name}.xyz', 'w') as f:
# #     # ITEM: TIMESTEP
# #     print(coord.shape)
# #     for i in tqdm(range(coord.shape[0]), desc = 'generating trajectory: '):
# #         f.write(f'ITEM: TIMESTEP\n{i}\n')
# #         f.write(f'ITEM: NUMBER OF ATOMS\n{coord.shape[1]}\n')
# #         f.write(f'ITEM: BOX BOUNDS xx yy zz pp pp pp\n{blo[0]:.8f} {bhi[0]}\n{blo[1]:.8f} {bhi[1]}\n{blo[2]:.8f} {bhi[2]}\n')
# #         f.write(f'ITEM: ATOMS id type x y z vx vy vz\n')
# #         for j in range(coord.shape[1]):
# #             f.write(f'{aid[j]:<5d}{atype[j]:3d}{coord[i,j,0]:11.6f}{coord[i,j,1]:11.6f}{coord[i,j,2]:11.6f}{velocity[i,j,0]:11.6f}{velocity[i,j,1]:11.6f}{velocity[i,j,2]:11.6f}\n')

# # coord
# # for j in range(coord.shape[1]):
# #     print(f'{aid[j]:<4d}{mol_idx[j]:<4d}{atype[j]:<3d}{q[j]:.5f} {coord[-1,j,0]:11.6f} {coord[-1,j,1]:11.6f} {coord[-1,j,2]:11.6f}\n', end = '')
# for j in range(coord.shape[1]):
#     print(f'{aid[j]:<4d}{velocity[-1,j,0]:11.6f}{velocity[-1,j,1]:11.6f}{velocity[-1,j,2]:11.6f}\n', end = '')